# Implementing cGEM-derived functional indices

### Researching on the community objective function

How is it defined? After inspecting ```print(cgem_gurobi.solver)```, apparently as a variable and contraint like this:

```text
 community_objective_equality: community_objective
   - 0.2 Growth__TARA_ARC_108_MAG_00174
   + 0.2 Growth__TARA_ARC_108_MAG_00174_reverse_e9527
   - 0.4 Growth__TARA_ARC_108_MAG_00083
   + 0.4 Growth__TARA_ARC_108_MAG_00083_reverse_f9d4e
   - 0.3 Growth__TARA_ARC_108_MAG_00080
   + 0.3 Growth__TARA_ARC_108_MAG_00080_reverse_ed394
   - 0.1 Growth__TARA_ARC_108_MAG_00201
   + 0.1 Growth__TARA_ARC_108_MAG_00201_reverse_1535b = 0
```

Which basically is just a weighted sum of all the biomass reactions of the community. The weights are the abundances of the models. The objective is to maximize the community biomass. Thus we can just include this constraint in the model when maximizing a given reaction.

In [1]:
from micom import load_pickle

cgem_hybrid = load_pickle("/home/robaina/Documents/NewAtlantis/microcom/test_hybrid/test_nf.pickle")
cgem_hybrid

Name,test_nf
Memory address,7f348e5261a0
Number of metabolites,7670
Number of reactions,11328
Number of genes,3097
Number of groups,0
Objective expression,1.0*community_objective
Compartments,"e__TARA_ARC_108_MAG_00174, p__TARA_ARC_108_MAG_00174, c__TARA_ARC_108_MAG_00174, m, e__TARA_ARC_108_MAG_00083, p__TARA_ARC_108_MAG_00083, c__TARA_ARC_108_MAG_00083, p__TARA_ARC_108_MAG_00080, c__TARA_ARC_108_MAG_00080, e__TARA_ARC_108_MAG_00080, p__TARA_ARC_108_MAG_00201, c__TARA_ARC_108_MAG_00201, e__TARA_ARC_108_MAG_00201"


In [3]:
cgem_hybrid.objective.expression

1.0*community_objective

### Add community ojective as a constraint

Adding a new pseudo reaction should work. The reaction will be a weighted sum of all the biomass reactions of the community. The weights are the abundances of the models. The objective is to maximize the community biomass. Thus we can just include this constraint in the model when maximizing a given reaction.

In [1]:
import numpy as np
from cobra import Model
from micom import Community
from optlang.symbolics import Zero, add
from optlang import Constraint
from typing import Dict


def update_community_growth_bounds(
        community_model: Community, lower_bound: float, upper_bound: float,
        obj_id: str = "community_objective") -> Community:
    """
    Changes the bounds of the community_growth variable in a MICOM Community model.

    Parameters:
    community_model (Community): The MICOM Community model.
    lower_bound (float): The new lower bound for the community objective variable.
    upper_bound (float): The new upper bound for the community objective variable.
    obj_id (str): The ID of the community objective variable. Default: 'community_objective'

    Returns:
    Community: The MICOM Community model with the bounds of the community objective variable changed.
    """
    community_obj = next(v for v in community_model.solver.variables if v.name == obj_id)
    community_obj.lb = lower_bound
    community_obj.ub = upper_bound
    return community_model


def add_weighted_flux_constraint(model: Model, reactions: Dict[str, float], lower_bound: float, upper_bound: float) -> Model:
    """
    Adds a custom constraint to a COBRApy model based on the weighted sum of specified reaction fluxes.

    Parameters:
    model (Model): The COBRApy model to which the constraint will be added.
    reactions (Dict[str, float]): A dictionary mapping reaction IDs to their respective weights in the constraint.
    lower_bound (float): The lower bound for the constraint.
    upper_bound (float): The upper bound for the constraint.

    Returns:
    None: The function modifies the model in-place by adding a new constraint.
    """
    expression = Zero
    for reaction_id, weight in reactions.items():
        expression += weight * model.reactions.get_by_id(reaction_id).flux_expression

    new_constraint = Constraint(expression, lb=lower_bound, ub=upper_bound)
    model.solver.add(new_constraint)
    return model

def construct_growth_dict(model: Community) -> Dict[str, float]:
    """
    Constructs a dictionary mapping growth reaction IDs to their abundances
    based on the abundances attribute in the COBRApy model.

    Parameters:
    model (Model): Micom's Community model which contains the abundances attribute.

    Returns:
    Dict[str, float]: A dictionary where keys are growth reaction IDs and values are abundances.
    """
    reactions = {}
    abundances_series = model.abundances  # Assuming model has an attribute 'abundances' which is a Pandas Series

    for taxon_id, abundance in abundances_series.items():
        reaction_id = f"Growth__{taxon_id}"
        reactions[reaction_id] = abundance
    return reactions


In [14]:
growth_reactions = construct_growth_dict(cgem_hybrid)
growth_reactions

{'Growth__TARA_ARC_108_MAG_00174': 0.2,
 'Growth__TARA_ARC_108_MAG_00083': 0.4,
 'Growth__TARA_ARC_108_MAG_00080': 0.3,
 'Growth__TARA_ARC_108_MAG_00201': 0.1}

In [2]:
cgem_hybrid.optimize()

,abundance,growth_rate,reactions,metabolites
compartments,,,,
TARA_ARC_108_MAG_00080,0.3,0.000000,2990,2013
TARA_ARC_108_MAG_00083,0.4,0.000000,2752,1825
TARA_ARC_108_MAG_00174,0.2,8.922886,3231,2085
TARA_ARC_108_MAG_00201,0.1,3.821992,1908,1300
medium,NaN,NaN,447,447


In [46]:
cgem_hybrid = update_community_growth_bounds(cgem_hybrid, 0.0, 1.8)
cgem_hybrid.optimize()

,abundance,growth_rate,reactions,metabolites
compartments,,,,
TARA_ARC_108_MAG_00080,0.3,6.0,2990,2013
TARA_ARC_108_MAG_00083,0.4,0.0,2752,1825
TARA_ARC_108_MAG_00174,0.2,0.0,3231,2085
TARA_ARC_108_MAG_00201,0.1,0.0,1908,1300
medium,NaN,NaN,447,447


In [ ]:
def extremize_flux(
        model: Model, reaction_id: str,
        growth_lower_bound: float, growth_upper_bound: float,
        direction: str = "max") -> float:
    """
    Adds a constraint to a COBRApy model to extremize the flux of a specified reaction.

    Parameters:
    model (Model): The COBRApy model to which the constraint will be added.
    reaction_id (str): The ID of the reaction whose flux will be extremized.
    growth_lower_bound (float): The lower bound for the community objective variable.
    growth_upper_bound (float): The upper bound for the community objective variable.
    direction (str): The direction of the optimization. Default: 'max'

    Returns:
    float: The optimal flux of the specified reaction.
    """
    model.objective = reaction_id
    model.objective_direction = direction
    model = update_community_growth_bounds(model, growth_lower_bound, growth_upper_bound)
    return model.optimize().objective_value


In [1]:
from cobra.flux_analysis.variability import flux_variability_analysis
from micom import load_pickle

cgem_hybrid = load_pickle("/home/robaina/Documents/NewAtlantis/microcom/test_hybrid/test_nf.pickle")
original_obj = cgem_hybrid.objective

In [18]:
reactions = ["EX_h2_m", "EX_co2_m", "EX_ac_m", "EX_tol_m"]
cgem_hybrid.abundances.loc["TARA_ARC_108_MAG_00080"] = 0.3

fva = flux_variability_analysis(
    cgem_hybrid, reaction_list=reactions,
    fraction_of_optimum=0.99, processes=1
    )
fva

/tmp/ipykernel_28119/2046264157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cgem_hybrid.abundances.loc["TARA_ARC_108_MAG_00080"] = 0.3


,minimum,maximum
EX_h2_m,-10.0,329.377197
EX_co2_m,-10.0,48.484207
EX_ac_m,0.0,0.000000
EX_tol_m,0.0,0.000000


In [16]:
cgem_hybrid.abundances.loc["TARA_ARC_108_MAG_00080"] = 0
fva = flux_variability_analysis(
    cgem_hybrid, reaction_list=reactions,
    fraction_of_optimum=0.99, processes=1
    )
fva

/tmp/ipykernel_28119/3399716721.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cgem_hybrid.abundances.loc["TARA_ARC_108_MAG_00080"] = 0


,minimum,maximum
EX_h2_m,-10.0,329.377197
EX_co2_m,-10.0,48.484207
EX_ac_m,0.0,0.000000


## Re-implement elasticity analysis based on FVA

## Sampling the flux space to compute elasticities

In [ ]:
import pandas as pd
import cobra

def sample_flux_space(
    model, n_samples: int = 1000, n_processes: int = None
    ) -> pd.DataFrame:
    """
    Sample the flux space of a self._model.

    Args:
        model (Model): The COBRApy model to sample.
        n_samples (int, optional): The number of samples to draw. Defaults to 1000.
        n_processes (int, optional): The number of processes to use. Defaults to None.

    Returns:
        DataFrame: A DataFrame containing the sampled fluxes.
    """
    if n_processes is None:
        n_processes = 2
    sol = model.optimize()
    flux_samples = cobra.sampling.sample(
        model, n_samples, method="achr", processes=n_processes
    )
    return flux_samples